In [1]:
import os, sys
import helper

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [4]:
tf.__version__

'1.2.1'

In [5]:
PAD = 0
EOS = 1

vocab_size = 10
input_embedding_size = 12

encoder_hidden_units = 20
decoder_hidden_units = encoder_hidden_units

In [6]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')
decoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_inputs')

In [7]:
decoder_inputs

<tf.Tensor 'decoder_inputs:0' shape=(?, ?) dtype=int32>

In [8]:
embeddings = tf.Variable(tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0), dtype=tf.float32)

In [9]:
embeddings

<tf.Variable 'Variable:0' shape=(10, 12) dtype=float32_ref>

In [10]:
encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)
decoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, decoder_inputs)

In [11]:
encoder_inputs_embedded

<tf.Tensor 'embedding_lookup:0' shape=(?, ?, 12) dtype=float32>

In [12]:
encoder_cell = tf.contrib.rnn.LSTMCell(encoder_hidden_units)

encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(
    encoder_cell, 
    encoder_inputs_embedded,
    dtype=tf.float32, 
    time_major=True,
)

In [13]:
encoder_final_state

LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_2:0' shape=(?, 20) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 20) dtype=float32>)

In [14]:
decoder_cell = tf.contrib.rnn.LSTMCell(decoder_hidden_units)

decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(
    decoder_cell, 
    decoder_inputs_embedded,
    initial_state=encoder_final_state,
    dtype=tf.float32, 
    time_major=True, 
    scope="plain_decoder",
)

In [15]:
decoder_outputs

<tf.Tensor 'plain_decoder/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, ?, 20) dtype=float32>

In [16]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, vocab_size)

In [42]:
decoder_logits

<tf.Tensor 'fully_connected/BiasAdd:0' shape=(?, ?, 10) dtype=float32>

In [18]:
decoder_prediction = tf.argmax(decoder_logits, 2)

In [43]:
decoder_targets

<tf.Tensor 'decoder_targets:0' shape=(?, ?) dtype=int32>

In [44]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(decoder_targets, depth=vocab_size, dtype=tf.float32),
    logits=decoder_logits,
)

loss = tf.reduce_mean(stepwise_cross_entropy)
train_op = tf.train.AdamOptimizer().minimize(loss)

In [50]:
sess.run(tf.global_variables_initializer())

In [45]:
batch_size = 100

In [46]:
batches = helper.random_sequences(length_from=3, 
                                   length_to=8,
                                   vocab_lower=2, 
                                   vocab_upper=10,
                                   batch_size=batch_size)

In [47]:
def next_feed():
    batch = next(batches)
    encoder_inputs_, _ = helper.batch(batch)
    decoder_targets_, _ = helper.batch(
        [(sequence) + [EOS] for sequence in batch]
    )
    decoder_inputs_, _ = helper.batch(
        [[EOS] + (sequence) for sequence in batch]
    )
    return {
        encoder_inputs: encoder_inputs_,
        decoder_inputs: decoder_inputs_,
        decoder_targets: decoder_targets_,
    }

In [48]:
loss_track = []

In [55]:
max_batches = 3001
batches_in_epoch = 1000

try:
    for batch in range(max_batches):
        fd = next_feed()
        _, state, l = sess.run([train_op, encoder_final_state, loss], fd)
        loss_track.append(l)
        if batch == 0 or batch % batches_in_epoch == 0:
            print('batch {}'.format(batch))
            print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            predict_ = sess.run(decoder_prediction, fd)
            for i, (inp, pred) in enumerate(zip(fd[encoder_inputs].T, predict_.T)):
                print('  sample {}:'.format(i + 1))
                print state[i]
                print('    input     > {}'.format(inp))
                print('    predicted > {}'.format(pred))
                if i >= 2:
                    break
            print()
except KeyboardInterrupt:
    print('training interrupted')

batch 0
  minibatch loss: 0.0407123975456
  sample 1:
[[-0.28191847 -0.4058125  -0.67629564 ..., -1.11924016 -0.31193531
   0.27761304]
 [-0.20310625 -1.12568533 -0.18539646 ..., -1.69361591 -0.02469572
   0.26339895]
 [-0.46359003 -1.01274765 -0.66975296 ..., -2.54661942 -0.40383697
  -0.37429926]
 ..., 
 [-0.04589035 -0.21897657 -0.48422003 ..., -1.54672217  0.530487
  -0.53486562]
 [-0.83836901 -0.82704931 -0.27398238 ..., -1.13414288 -0.24166071
  -0.53352398]
 [-0.56390798 -1.22604263 -0.67457879 ..., -1.60757065 -0.50900394
  -0.08410753]]
    input     > [5 3 3 4 9 0 0 0]
    predicted > [5 3 3 4 9 1 0 0 0]
  sample 2:
[[-0.12828502 -0.29245752 -0.47119397 ..., -0.65907085 -0.24277306
   0.10708448]
 [-0.09616902 -0.66488224 -0.1400418  ..., -0.76927173 -0.02078826
   0.08343641]
 [-0.18774366 -0.61240757 -0.43981749 ..., -0.83982736 -0.31294605
  -0.1195562 ]
 ..., 
 [-0.02284209 -0.15970922 -0.37164119 ..., -0.68907875  0.38851413
  -0.19159004]
 [-0.27703723 -0.53385979 -0.24

IndexError: tuple index out of range